# Data Crawler - Valorant

## Libs

In [31]:
import pandas as pd
import requests
import json
import pathlib
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

## Initial Settings

In [2]:
abs_path = pathlib.Path.cwd().parent

In [24]:
player_path = pathlib.Path(abs_path).joinpath('data').joinpath('players').joinpath('players.csv')
matches_path = pathlib.Path(abs_path).joinpath('data').joinpath('matches').joinpath('matches.csv')
weapons_path = pathlib.Path(abs_path).joinpath('data').joinpath('weapons').joinpath('weapons.csv')
webdriver_path = pathlib.Path(abs_path).joinpath('chromedriver.exe')

## Players

In [5]:
def get_top_players(servers_list) -> str:
    '''
        This function's mission is to get all players that are ranked in the top 500 of a server list.
        :param [list] server_list: A variable that receives a server list. For example: 'kr', 'eu', 'na', 'br', 'latam', 'ap'.
        :return [pd.dataframe] data: A variable that receives a string with the structure of a json. This string contains the summarized information of a player that who's in top 500.
    '''
    path_write = 'raw/rank/top500/all_servers/'
    data = []
    df = pd.DataFrame()
    file_format = '.csv'

    for server in servers_list:
            
        for page in range(1, 5):

            response = requests.get('https://val.dakgg.io/api/v1/leaderboards/{}/aca29595-40e4-01f5-3f35-b1b3d304c96e?page={}&tier=top500'.format(server, page))
            data_aux = response.json()
            data.append(data_aux)

        data_upload = str(data)


    df = pd.DataFrame(data)
    df = pd.json_normalize(json.loads(df.to_json(orient='records'))).explode('leaderboards')
    df = pd.json_normalize(json.loads(df.to_json(orient='records')))

    df['leaderboards.full_nickname'] = (df['leaderboards.gameName'].map(str) + '%23' + df['leaderboards.tagLine'].map(str))
    df.to_csv(player_path, index=False)
            
    return df

In [7]:
get_top_players(['br', 'na', 'latam', 'eu', 'kr', 'ap'])

,meta.shard,meta.season,meta.page,meta.perPage,meta.totalCount,meta.updatedAt,leaderboards.puuid,leaderboards.gameName,leaderboards.tagLine,leaderboards.leaderboardRank,leaderboards.rankedRating,leaderboards.numberOfWins,leaderboards.competitiveTier,leaderboards.stat.winRatio,leaderboards.stat.avgScore,leaderboards.stat.headshotRatio,leaderboards.stat.mostCharacters,leaderboards.full_nickname
0,br,aca29595-40e4-01f5-3f35-b1b3d304c96e,1,100,500,2022-12-19T20:00:33.000Z,JA8zenU_bSaw5meP7nHt5_Z7m5LFs96XGyJi88iIwkFQG8...,artzin,psz,1,1122,199,27,0.611,262.0,0.274,[{'characterId': '22697a3d-45bf-8dd7-4fec-84a9...,artzin%23psz
1,br,aca29595-40e4-01f5-3f35-b1b3d304c96e,1,100,500,2022-12-19T20:00:33.000Z,eD6q4kd8wPUPxN9aqU8eXYpCJrviTtPdgPVIak76ahdgdO...,TBK anao sato,lukxo,2,1114,112,27,0.570,249.0,0.341,[{'characterId': '22697a3d-45bf-8dd7-4fec-84a9...,TBK anao sato%23lukxo
2,br,aca29595-40e4-01f5-3f35-b1b3d304c96e,1,100,500,2022-12-19T20:00:33.000Z,1q93nnIL_LTjr-30olVrL1T5PBFkTEe-UFL90_Nv__3L1h...,LOUD cauanzin,2005,3,1059,191,27,0.644,228.0,0.266,[{'characterId': '320b2a48-4d9b-a075-30f1-1f93...,LOUD cauanzin%232005
3,br,aca29595-40e4-01f5-3f35-b1b3d304c96e,1,100,500,2022-12-19T20:00:33.000Z,GoKHN_B78EbVlhaJZ30nR8-68Pd5IIbTp-u2Bslz_7cnSA...,kiNgg,neave,4,1056,91,27,0.586,242.0,0.280,[{'characterId': '707eab51-4836-f488-046a-cda6...,kiNgg%23neave
4,br,aca29595-40e4-01f5-3f35-b1b3d304c96e,1,100,500,2022-12-19T20:00:33.000Z,qDV-PKIh8KepwYOmC6CkSyZT_h1w3a7YnkgA-PTQpjxnic...,ODK Tisora,zzz,5,1056,107,27,0.644,282.0,0.275,[{'characterId': '22697a3d-45bf-8dd7-4fec-84a9...,ODK Tisora%23zzz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,ap,aca29595-40e4-01f5-3f35-b1b3d304c96e,4,100,500,2022-12-19T20:01:28.000Z,T_VlFbYvZims_pxLtRr3Hx4F9U1bBfYWMPCihqvH4UlF1M...,ROSETISLAV,SCH,396,567,134,27,0.663,258.0,0.156,[{'characterId': '22697a3d-45bf-8dd7-4fec-84a9...,ROSETISLAV%23SCH
2396,ap,aca29595-40e4-01f5-3f35-b1b3d304c96e,4,100,500,2022-12-19T20:01:28.000Z,77qu5Y53fWq_AEfXDwo1qq8iNvC1effKno5Pfzpyst0bDb...,Only1,Dhana,397,566,196,27,0.510,229.0,0.282,[{'characterId': 'add6443a-41bd-e414-f6ad-e58d...,Only1%23Dhana
2397,ap,aca29595-40e4-01f5-3f35-b1b3d304c96e,4,100,500,2022-12-19T20:01:28.000Z,NaN,NaN,NaN,398,566,41,27,NaN,NaN,NaN,NaN,nan%23nan
2398,ap,aca29595-40e4-01f5-3f35-b1b3d304c96e,4,100,500,2022-12-19T20:01:28.000Z,odtzTI2NeznNVIm1kiP-M1C8P94_GZLdEg749G-pBuHasw...,ExTreMisTz,3736,399,566,208,27,0.567,186.0,0.258,[{'characterId': '320b2a48-4d9b-a075-30f1-1f93...,ExTreMisTz%233736


## Matches

In [36]:
def get_matches(players_list) -> str:
    '''
        This function's mission is to get a summary report of all the last 200 matches of a specific player.
        :param [list] players_list: A variable that receives a players list. For example: ['NaraKa%232299','NakaRa%233265','RayzenSama%236999'].
        :return [str] data_pre: A variable that receives a string with the structure of a json. This string contains the summarized information of a player who's in the top 500.
    '''

    driver = webdriver.Chrome(webdriver_path)
    driver = webdriver.Chrome(ChromeDriverManager().install())
    
    
    data = []
    for player in players_list:
        

        for page in range(1,10):

            
            driver.get('https://api.tracker.gg/api/v2/valorant/standard/matches/riot/{}?type=competitive&next={}'.format(player, page))
            data_pre = driver.find_element('xpath', '//pre').text
            data_json = json.loads(data_pre)
            if 'data' in data_json.keys():
                matches = data_json['data']['matches']
                for match in matches:
                    dataframe_row = dict()
                    dataframe_row['player'] = player
                    dataframe_row['matchId'] = match['attributes']['id']
                    dataframe_row['mapId'] = match['attributes']['mapId']
                    dataframe_row['modeId'] = match['attributes']['modeId']
                    dataframe_row['modeKey'] = match['metadata']['modeKey']
                    dataframe_row['modeName'] = match['metadata']['modeName']
                    dataframe_row['modeImageUrl'] = match['metadata']['modeImageUrl']
                    dataframe_row['modeMaxRounds'] = match['metadata']['modeMaxRounds']
                    dataframe_row['isAvailable'] = match['metadata']['isAvailable']
                    dataframe_row['timestamp'] = match['metadata']['timestamp']
                    dataframe_row['metadataResult'] = match['metadata']['result']
                    dataframe_row['map'] = match['metadata']['map']
                    dataframe_row['mapName'] = match['metadata']['mapName']
                    dataframe_row['mapImageUrl'] = match['metadata']['mapImageUrl']
                    dataframe_row['seasonName'] = match['metadata']['seasonName']
                    dataframe_row['userId'] = match['segments'][0]['attributes']['platformUserIdentifier']
                    dataframe_row['hasWon'] = match['segments'][0]['metadata']['hasWon']
                    dataframe_row['result'] = match['segments'][0]['metadata']['result']
                    dataframe_row['agentName'] = match['segments'][0]['metadata']['agentName']
                    for stat_name, stat_value in match['segments'][0]['stats'].items():
                        dataframe_row[f"{stat_name}Value"] = stat_value['value']
                        dataframe_row[f"{stat_name}DisplayValue"] = stat_value['displayValue']
                        dataframe_row[f"{stat_name}DisplayType"] = stat_value['displayType']
                    data.append(dataframe_row)

    df = pd.DataFrame.from_dict(data)
    df.to_csv(matches_path, index=False)

    driver.quit()

    return df

In [33]:
player_list = pd.read_csv(player_path)

In [37]:
get_matches(player_list['leaderboards.full_nickname'])

C:\Users\alexa\AppData\Local\Temp\ipykernel_9000\3699045190.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(webdriver_path)
C:\Users\alexa\AppData\Local\Temp\ipykernel_9000\3699045190.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
def get_weapons(players_list) -> str:
    '''
        This function's mission is to get a summary report of all the last 200 matches of a specific player.
        :param [list] players_list: A variable that receives a players list. For example: ['NaraKa%232299','NakaRa%233265','RayzenSama%236999'].
        :return [str] data_pre: A variable that receives a string with the structure of a json. This string contains the summarized information of a player who's in the top 500.
    '''
    driver = webdriver.Chrome()
    data = []
    for player in players_list:
        
        driver.get('https://api.tracker.gg/api/v2/valorant/standard/profile/riot/{}/segments/weapon?playlist=competitive&seasonId=default'.format(player))
        data_pre = driver.find_element('xpath', '//pre').text
        data_json = json.loads(data_pre)
        if 'data' in data_json.keys():
            weapons = data_json['data']
            for weapon in weapons:
                dataframe_row = dict()
                dataframe_row['player'] = player
                dataframe_row['weaponName'] = weapon['attributes']['key']
                for stat_name, stat_value in weapon['stats'].items():
                    dataframe_row[f"{stat_name}Value"] = stat_value['value']
                    dataframe_row[f"{stat_name}DisplayValue"] = stat_value['displayValue']
                    dataframe_row[f"{stat_name}DisplayType"] = stat_value['displayType']
                data.append(dataframe_row)

    df = pd.DataFrame.from_dict(data)
    df.to_csv(weapons_path, index=False)
    driver.quit()

    return df